<!--html_preserve-->
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=UA-130562131-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-130562131-1');
</script>
<!--/html_preserve-->


In [ ]:
knitr::include_graphics("https://slcladal.github.io/images/uq1.jpg")



# Introduction

This tutorial introduces regular expressions and how they can be used when working with language data. The entire R markdown document for the sections below can be downloaded [here](https://slcladal.github.io/regex.Rmd).

How can you search texts for complex patterns or combinations of patterns? This question will answered in this tutorial and at the end you will be able to perform very complex searches yourself. The key concept of this tutorial is that of a regular expression. A regular expression (in short also called *regex* or *regexp*) is a special sequence of characters (or string) for describing a search pattern. You can think of regular expressions as very powerful combinations of wildcards or as wildcards on steroids. 

If you would like to get deeper into regular expressions, I can recommend @friedl2006mastering and, in particular, chapter 17 of @peng2016r for further study (although the latter uses base R rather than tidyverse functions, but this does not affect the utility of the discussion of regular expressions in any major or meaningful manner). Also, [here](https://rstudio.com/wp-content/uploads/2016/09/RegExCheatsheet.pdf) is a so-called *cheatsheet* about regular expressions written by  Ian Kopacka and provided by RStudio.

**Preparation and session set up**

This tutorial is based on R. If you have not installed R or are new to it, you will find an introduction to and more information how to use R [here](https://slcladal.github.io/Intror.html). For this tutorials, we need to install certain *packages* from an R *library* so that the scripts shown below are executed without errors. Before turning to the code below, please install the packages by running the code below this paragraph. If you have already installed the packages mentioned below, then you can skip ahead and ignore this section. To install the necessary packages, simply run the following code - it may take some time (between 1 and 5 minutes to install all of the libraries so you do not need to worry if it takes some time).


In [ ]:
# set options
options(stringsAsFactors = F)
# load packages
library(tidyverse)


In [ ]:
library(kableExtra)
library(knitr)
library(DT)


Once you have installed R-Studio and have initiated the session by executing the code shown above, you are good to go.

# Getting started with Regular Expressions

To put regular expressions into practice, we need some text that we will perform out searches on. In this tutorial, we will use texts from wikipedia about grammar.


In [ ]:
# read in first text
text1 <- readLines("https://slcladal.github.io/data/testcorpus/linguistics02.txt")
et <-  paste(text1, sep = " ", collapse = " ")
# inspect example text
et


In addition, we will split the example text into words to have another resource we can use to understand regular expressions



In [ ]:
# split example text
set <- str_split(et, " ") %>%
  unlist()
# inspect
head(set)


Before we delve into using regular expressions, we will have a look at the regular expressions that can be used in R and also check what they stand for.

There are three basic types of regular expressions:

* regular expressions that stand for individual symbols and determine frequencies

* regular expressions that stand for classes of symbols

* regular expressions that stand for structural properties

The regular expressions below show the first type of regular expressions, i.e. regular expressions that stand for individual symbols and determine frequencies.


In [ ]:
symbols1 <- c(fixed("?"), fixed("*"), fixed("+"), "{n}", "{n,}", "{n,m}")
explanation1 <- c("The preceding item is optional and will be matched at most once", "The preceding item will be matched zero or more times", "The preceding item will be matched one or more times", "The preceding item is matched exactly n times", "The preceding item is matched n or more times", "The preceding item is matched at least n times, but not more than m times")
example1 <- c("walk[a-z]? = walk, walks", 
             "walk[a-z]* = walk, walks, walked, walking", 
             "walk[a-z]+ = walks, walked, walking", 
             "walk[a-z]{2} = walked", 
             "walk[a-z]{2,} = walked, walking", 
             "walk[a-z]{2,3} = walked, walking")
df_regex <- data.frame(symbols1, explanation1, example1)
colnames(df_regex) <- c("RegEx Symbol/Sequence", "Explanation", "Example")
datatable(df_regex, rownames = FALSE, filter="none", options = list(pageLength = 6, scrollX=T), caption = "Regular expressions that stand for individual symbols and determine frequencies.")


The regular expressions below show the second type of regular expressions, i.e. regular expressions that stand for classes of symbols.



In [ ]:
symbols2 <- c("[ab]", "[a-z]", "[AB]", "[A-Z]", "[12]", "[0-9]", "[:digit:]", "[:lower:]", "[:upper:]", "[:alpha:]", "[:alnum:]", "[:punct:]", "[:graph:]", "[:blank:]", "[:space:]", "[:print:]")
explanations2 <- c("lower case a and b",
                   "all lower case characters from a to z",
                   "upper case a and b",
                   "all upper case characters from A to Z",
                   "digits 1 and 2", 
                   "digits: 0 1 2 3 4 5 6 7 8 9",
                   "digits: 0 1 2 3 4 5 6 7 8 9", 
                   "lower case characters: a–z", 
                   "upper case characters: A–Z", 
                   "alphabetic characters: a–z and A–Z", 
                   "digits and alphabetic characters", 
                   "punctuation characters: . , ; etc.", 
                   "graphical characters: [:alnum:] and [:punct:]", 
                   "blank characters: Space and tab", 
                   "space characters: Space, tab, newline, and other space characters", 
                   "printable characters: [:alnum:], [:punct:] and [:space:]")
df_regex <- data.frame(symbols2, explanations2)
colnames(df_regex) <- c("RegEx Symbol/Sequence", "Explanation")
datatable(df_regex, rownames = FALSE, filter="none", options = list(pageLength = 16, scrollX=T), caption = "Regular expressions that stand for classes of symbols.")


The regular expressions that denote classes of symbols are enclosed in `[]` and `:`. The last type of regular expressions, i.e. regular expressions that stand for structural properties are shown below.



In [ ]:
symbols3 <- c(fixed("\\\\w"), fixed("\\\\W"), fixed("\\\\s"), fixed("\\\\S"), 
              fixed("\\\\d"), fixed("\\\\D"), fixed("\\\\b"), fixed("\\\\B"), 
              fixed("<"), fixed(">"), fixed("^"), fixed("$"))
explanations3 <- c("Word characters: [[:alnum:]_]",
                   "No word characters: [^[:alnum:]_]",
                   "Space characters: [[:blank:]]",
                   "No space characters: [^[:blank:]]",
                   "Digits: [[:digit:]]",
                   "No digits: [^[:digit:]]",
                   "Word edge",
                   "No word edge",
                   "Word beginning",
                   "Word end",
                   "Beginning of a string",
                   "End of a string")
df_regex <- data.frame(symbols3, explanations3)
colnames(df_regex) <- c("RegEx Symbol/Sequence", "Explanation")
datatable(df_regex, rownames = FALSE, filter="none", options = list(pageLength = 13, scrollX=T), caption = "Regular expressions that stand for structural properties.")


# Practice

In this section, we will explore how to use regular expressions.

Show all words in the split example text that contain `a` or `n`.


In [ ]:
set[str_detect(set, "[an]")]



Show all words in the split example text that begin with a lower case `a`.



In [ ]:
set[str_detect(set, "^a")]



Show all words in the split example text that end in a lower case `s`.



In [ ]:
set[str_detect(set, "s$")]



Show all words in the split example text in which there is an `e`, then any other character, and than another `n`.



In [ ]:
set[str_detect(set, "e.n")]



Show all words in the split example text in which there is an `e`, then two other characters, and than another `n`.



In [ ]:
set[str_detect(set, "e.{2,2}n")]



Show all words that consist of exactly three alphabetical characters in the split example text.



In [ ]:
set[str_detect(set, "^[:alpha:]{3,3}$")]



Show all words that consist of six or more alphabetical characters in the split example text.



In [ ]:
set[str_detect(set, "^[:alpha:]{6,}$")]



Replace all lower case `a`s with upper case `E`s in the example text.



In [ ]:
str_replace_all(et, "a", "E")



Remove all non-alphabetical characters in the split example text.



In [ ]:
str_remove_all(set, "\\W")



Remove all white spaces in the example text.



In [ ]:
str_remove_all(et, " ")



**Highlighting patterns**

We use the `str_view` and `str_view_all` functions to show the occurrences of regular expressions in the example text.

To begin with, we match an exactly defined pattern (`ang`).


In [ ]:
str_view_all(et, "ang")



Now, we include . which stands for any symbol (except a new line symbol).



In [ ]:
str_view_all(et, ".n.")



# Citation & Session Info {-}

Schweinberger, Martin. `r format(Sys.time(), '%Y')`. *Regular Expressions in R*. Brisbane: The University of Queensland. url: https://slcladal.github.io/regex.html (Version `r format(Sys.time(), '%Y.%m.%d')`).


In [ ]:
@manual{schweinberger`r format(Sys.time(), '%Y')`regex,
  author = {Schweinberger, Martin},
  title = {Regular Expressions in R},
  note = {https://slcladal.github.io/regex.html},
  year = {`r format(Sys.time(), '%Y')`},
  organization = {The University of Queensland, School of Languages and Cultures},
  address = {Brisbane},
  edition = {`r format(Sys.time(), '%Y.%m.%d')`}
}


In [ ]:
sessionInfo()



***

[Back to top](#introduction)

[Back to HOME](https://slcladal.github.io/index.html)

***

# References {-}
